# Tutorial and cookbook

By [Allison Parrish](http://www.decontextualize.com/)

Draft version!

This notebook shows you how to use [Pincelate](https://pincelate.readthedocs.io/) and how to do some interesting things with it.

Pincelate is a Python library that provides a simple interface for a machine learning model that can sound out English words and spell English words based on how they sound. "Sounding out" here means converting letters ("orthography") to sounds ("phonemes"), and "spelling" means converting sounds to letters (phonemes to orthography). The model is trained on the [CMU Pronouncing Dictionary](http://www.speech.cs.cmu.edu/cgi-bin/cmudict), which means it generally sounds words out as though speaking "standard" North American English, and spells words according to "standard" North American English rules (at least as far as the model itself is accurate).

## Preliminaries

Loading various required modules, plus the language model and Pincelate. To run these experiments, you'll need to install Pincelate. Type the following at a command prompt:

    pip install tensorflow  # or tensorflow-gpu
    pip install pincelate
    
(Installing Pincelate will also install Pronouncing, which we'll use at various points in the experiments below.)

Other libraries you'll need for this notebook: `numpy` and `scipy`. If you're using Anaconda, you already have these libraries. If not, install them like so:

    pip install numpy scipy
    
Importing numpy and Pronouncing:

In [1]:
import numpy as np
import pronouncing as pr

Now import Pincelate and instantiate a Pincelate object. (This will load the pre-trained model provided with the package.)

In [2]:
from pincelate import Pincelate

Using TensorFlow backend.


In [3]:
pin = Pincelate()

Later in the notebook, I'm going to use some of Jupyter Notebook's interactive features, so I'll import the libraries here:

In [4]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive_output, Layout, HBox, VBox

## Sounding out and spelling

The CMU Pronouncing Dictionary provides a database of tens of thousands of English words along with their pronunciations. I made a Python library called [Pronouncing](https://github.com/aparrish/pronouncingpy) to make it easier to look up words in dictionary. Here's how it works. To get the pronunciation of a word:

In [5]:
pr.phones_for_word("alphabet")[0]

'AE1 L F AH0 B EH2 T'

The CMU Pronouncing Dictionary provides pronunciations as a list of phonemes in a phonetic transcription scheme called [Arpabet](https://en.wikipedia.org/wiki/ARPABET), in which each unique sound in English is given a different symbol.

If you want to find words that have a particular pronunciation, you can look them up in the CMU Pronouncing Dictionary like so:

In [6]:
pr.search("^F L AW1 ER0$")

['flour', 'flower']

That all seems pretty straightforward! The problem arises when you want to spell a word that *isn't* in the CMU Pronouncing Dictionary. You'll get an error:

In [7]:
pr.phones_for_word("mimsy")[0]

IndexError: list index out of range

Likewise, if you've just invented a new word and have a pronunciation in mind, the CMU Pronouncing Dictionary won't be able to help you spell it:

In [8]:
pr.search("^B L AH1 R F$")

[]

This is where Pincelate comes in handy. Pincelate's machine learning model can provide phonemes for words that aren't in the CMU Pronouncing Dictionary, and produce plausible spellings of arbitrary sequences of phonemes. To sound out a word, use the `.soundout()` method:

In [9]:
pin.soundout("mimsy")

['M', 'IH1', 'N', 'S', 'IY0']

... and to produce a plausible spelling for a word whose sounds you just made up, use the `.spell()` method, passing it a list of Arpabet phonemes:

In [10]:
pin.spell(['B', 'L', 'AH1', 'R', 'F'])

'blear'

It's important to note that Pincelate's `.soundout()` method will *only* work with letters that appear the CMU Pronouncing Dictionary's vocabulary. (You need to use lowercase letters only.) So the following will throw an error:

In [11]:
pin.spell("étui")

KeyError: 'é'

### Example: phoneme frequency analysis

Using Pincelate's model, we can do phonetic analysis on texts, even texts that contain words that aren't in the CMU Pronouncing Dictionary. For example, let's find out what the most common phonemes are in Lewis Carroll's "Jabberwocky." Here's the full text:

In [12]:
text = """
'Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.

"Beware the Jabberwock, my son!
      The jaws that bite, the claws that catch!
Beware the Jubjub bird, and shun
      The frumious Bandersnatch!"

He took his vorpal sword in hand;
      Long time the manxome foe he sought---
So rested he by the Tumtum tree
      And stood awhile in thought.

And, as in uffish thought he stood,
      The Jabberwock, with eyes of flame,
Came whiffling through the tulgey wood,
      And burbled as it came!

One, two! One, two! And through and through
      The vorpal blade went snicker-snack!
He left it dead, and with its head
      He went galumphing back.

"And hast thou slain the Jabberwock?
      Come to my arms, my beamish boy!
O frabjous day! Callooh! Callay!"
      He chortled in his joy.

'Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.
"""

First, parse the text into words and convert them to lower case:

In [13]:
import re
words = [item.lower() for item in re.findall(r"\b(\w+)\b", text)]

Here's a random sample of the words just to ensure that we've got what we wanted:

In [14]:
import random
random.sample(words, 10)

['the',
 'and',
 'so',
 'frumious',
 'thou',
 'awhile',
 'o',
 'tumtum',
 'one',
 'wabe']

Now, we'll use `.soundout()` to get a list of phonemes for each item, and feed them to a `Counter()` object:

In [15]:
from collections import Counter
phoneme_count = Counter()
for word in words:
    phoneme_count.update(pin.soundout(word))

And now print out the most common phonemes:

In [16]:
phoneme_count.most_common(12)

[('N', 51),
 ('IH1', 43),
 ('T', 36),
 ('S', 36),
 ('AE1', 36),
 ('R', 34),
 ('D', 32),
 ('TH', 27),
 ('EH1', 27),
 ('L', 24),
 ('B', 18),
 ('M', 17)]

For reference, the following cell calculates the most common phonemes in all of the CMU Pronouncing Dictionary:

In [17]:
cmu_phoneme_count = Counter()
for word, phones in pr.pronunciations:
    cmu_phoneme_count.update(phones.split())
cmu_phoneme_count.most_common(12)

[('AH0', 63133),
 ('N', 61234),
 ('S', 50432),
 ('L', 49963),
 ('T', 49074),
 ('R', 46468),
 ('K', 43077),
 ('D', 32558),
 ('IH0', 30198),
 ('M', 29741),
 ('Z', 28217),
 ('ER0', 23954)]

We could do a more formal analysis and make claims about how Lewis Carroll's *Jabberwocky* differs significantly from typical English from a phonetic standpoint, but just from a quick look we can see that "Jabberwocky" is heavy on the `AE1`s (i.e., the vowel sound in "hand") and `B`s.

### Example: Spelling words from random phonemes

Having just counted up all of the phonemes in the CMU Pronouncing Dictionary, we can now invent somewhat plausible neologisms by drawing phonemes at random according to their frequency and gluing them together. ("Neologism" is a fancy word for "made-up word.") The following code normalizes the phoneme frequencies so we can use them in numpy's `np.random.choice` function:

In [18]:
all_phonemes = list(cmu_phoneme_count.keys())
phoneme_frequencies = np.array(list(cmu_phoneme_count.values()), dtype=np.float32)
phoneme_frequencies /= phoneme_frequencies.sum()

And then this function will return a random neologism, created from phonemes drawn at random based on their frequency in English words:

In [19]:
def neologism_phonemes():
    return [np.random.choice(all_phonemes, p=phoneme_frequencies)
            for item in range(random.randrange(3,10))]

Here's a handful, just to get a taste:

In [20]:
for i in range(5):
    print(neologism_phonemes())

['T', 'IH1', 'L', 'N', 'T', 'N', 'N', 'T', 'ER0']
['AA1', 'V', 'OW1', 'AH0']
['AW1', 'G', 'G', 'ER1']
['AE2', 'N', 'K']
['EH1', 'B', 'L', 'L', 'IH0']


That's all well and good! Try sounding out some of these on your own (consult the [Arpabet](https://en.wikipedia.org/wiki/ARPABET) table to find the English sound corresponding to each symbol).

But how do you *spell* these neologisms? Why, with Pincelate's `.spell()` method of course:

In [21]:
pin.spell(neologism_phonemes())

'richale'

Here's a for loop that generates neologisms and prints them along with their spellings:

In [22]:
for i in range(12):
    phonemes = neologism_phonemes()
    print(pin.spell(phonemes), phonemes)

roughler ['R', 'AH0', 'ER0', 'L', 'R', 'L', 'V', 'ER1']
earole ['IH2', 'R', 'L', 'OW1', 'T']
eaist ['IY2', 'IY0', 'HH', 'D', 'D']
aibes ['AE1', 'IH0', 'B', 'S']
iessier ['IY0', 'IY0', 'S', 'IY2', 'ER0']
stemen ['S', 'T', 'EH1', 'M', 'T', 'S', 'N']
chonce ['K', 'N', 'AA1', 'K', 'K', 'AH0', 'R', 'S']
nlucke ['N', 'L', 'T', 'AH1', 'K']
mitson ['M', 'T', 'IH1', 'D', 'S', 'IH1', 'K', 'N']
erte ['ER0', 'T', 'R']
briche ['B', 'K', 'G', 'HH', 'ER1', 'IH1']
ailear ['EY1', 'L', 'EY1', 'B']


## Phoneme features

The examples above use the phoneme as the basic unit of English phonetics. But each phoneme itself has characteristics, and many phonemes have characteristics in common. For example, the phoneme `/B/` has the following characteristics:

* *bilabial*: you put your lips together when you say it
* *stop*: airflow from the lungs is completely obstructed
* *voiced*: your vocal cords are vibrating while you say it

The phoneme `/P/` shares two out of three of these characteristics (it's *bilabial* and a *stop*, but is not voiced). The phoneme `/AE/`, on the other hand, shares *none* of these characteristics. Instead, it has these characteristics:

* *vowel*: your mouth doesn't stop or occlude airflow when making this sound
* *low*: your tongue is low in the mouth
* *front*: your tongue is advanced forward in the mouth
* *unrounded*: your lips are not rounded

These characteristics of phonemes are traditionally called "features." You can look up the features for particular phonemes using the `phone_feature_map` variable in Pincelate's `featurephone` module:

In [23]:
from pincelate.featurephone import phone_feature_map

For example, to get the features for the vowel `/UW/` (vowel sound in "toot"):

In [24]:
phone_feature_map['UW']

('hgh', 'bck', 'rnd', 'vwl')

The features are referred to here with short three-letter abbreviations. Here's a full list:

* `alv`: alveolar
* `apr`: approximant
* `bck`: back
* `blb`: bilabial
* `cnt`: central
* `dnt`: dental
* `fnt`: front
* `frc`: fricative
* `glt`: glottal
* `hgh`: high
* `lat`: lateral
* `lbd`: labiodental
* `lbv`: labiovelar
* `lmd`: low-mid
* `low`: low
* `mid`: mid
* `nas`: nasal
* `pal`: palatal
* `pla`: palato-alveolar
* `rnd`: rounded
* `rzd`: rhoticized
* `smh`: semi-high
* `stp`: stop
* `umd`: upper-mid
* `unr`: unrounded
* `vcd`: voiced
* `vel`: velar
* `vls`: voiceless
* `vwl`: vowel

Additionally, there are two special phoneme features:

* `beg`: beginning of word
* `end`: end of word

... which are found and the beginnings and endings of words.

Internally, Pincelate's model operates on these *phoneme features*, instead of directly on whole phonemes. This allows the model to capture and predict underlying similarities between phonemes.

Pincelate's `.phonemefeatures()` method works a lot like `.spell()`, except instead of returning a list of phonemes, it returns a [numpy](https://numpy.org/) array of *phoneme feature probabilities*. This array has one row for each predicted phoneme, and one column for the probability (between 0 and 1) of a phoneme feature being a component of each phoneme. To illustrate, here I get the feature array for the word `cat`:

In [25]:
cat_feats = pin.phonemefeatures("cat")

This array has the following shape:

In [26]:
cat_feats.shape

(5, 32)

... which tells us that there are five predicted phonemes. (The `32` is the total number of possible features.) The word `cat`, of course, has only three phonemes (`/K AE T/`)—the extra two are the special "beginning of the word" and "end of the word" phonemes at the beginning and end, respectively.

### Examining predicted phoneme features

Let's look at the feature probabilities for the first phoneme (after the special "beginning of the word" token at index 0):

In [27]:
cat_feats[1]

array([4.47422266e-04, 1.18613243e-05, 4.32133675e-06, 0.00000000e+00,
       4.28557396e-05, 1.25169754e-06, 0.00000000e+00, 2.98023224e-08,
       9.17911530e-06, 2.17527151e-04, 4.31716442e-04, 2.38418579e-07,
       0.00000000e+00, 6.36577606e-05, 2.98023224e-08, 2.05636024e-06,
       2.62558460e-05, 1.49011612e-07, 3.12924385e-06, 7.15255737e-07,
       1.17987394e-04, 2.08616257e-07, 0.00000000e+00, 4.08291817e-06,
       9.99740005e-01, 1.83761120e-04, 3.27825546e-07, 4.73856926e-06,
       1.52707100e-04, 9.99939680e-01, 9.99865890e-01, 9.04798508e-05])

You can look up the index in this array associated with a particular phoneme feature using Pincelate's `.featureidx()` method:

In [28]:
cat_feats[1][pin.featureidx('vel')]

0.9999396800994873

This tells us that the `vel` (velar) feature for this phoneme is predicted with almost 100% probability—which makes sense, since the phoneme we'd anticipate—`/K/` is a voiceless velar stop.

The following bit of code steps through each row in this array and prints out the phoneme features with the highest probability in that row, using numpy's `argsort` function:

In [29]:
def idxfeature(pin, idx):
    return pin.orth2phon.target_vocab[idx]
for i, phon in enumerate(cat_feats):
    print("phoneme", i)
    for idx in np.argsort(phon)[::-1][:5]:
        print(idxfeature(pin, idx), phon[idx])
    print()

phoneme 0
beg 1.0
vwl 0.0
vls 0.0
apr 0.0
bck 0.0

phoneme 1
vel 0.9999396800994873
vls 0.9998658895492554
stp 0.9997400045394897
alv 0.0004474222660064697
glt 0.0004317164421081543

phoneme 2
vwl 0.9994708895683289
unr 0.9978153109550476
str 0.9701219201087952
low 0.9265567064285278
fnt 0.8983818888664246

phoneme 3
alv 0.9887728095054626
vls 0.9821089506149292
stp 0.9477603435516357
frc 0.11646848917007446
vcd 0.0205477774143219

phoneme 4
end 0.9521546363830566
vwl 0.23826062679290771
unr 0.17280638217926025
cnt 0.07723551988601685
fnt 0.059767305850982666



### Example: Distinctive phoneme features in lines of poetry

Poems are often organized into lines. In the following example, I look at our example poem ("Jabberwocky", included this notebook above) and try to figure out what makes each line of the poem *phonetically distinct* from the other lines.

To do this, we need a baseline of phoneme feature frequency in the entire text. As a measure of this, I'm just going to calculate phoneme feature probabilities for every word in the poem (using the `words` list defined above), add them up, and normalize by the number of words in the poem. The code in the following cell uses `.phonemefeatures()` for each word and then numpy's `concatenate()` function to stack them into one big array:

In [30]:
word_feats = np.concatenate([pin.phonemefeatures(word) for word in words])

In [31]:
word_feats.shape

(912, 32)

Then we normalize by the length of the array:

In [32]:
word_feats_normal = word_feats.sum(axis=0) / word_feats.shape[0]

The following cell prints out the top ten phoneme features in the text:

In [33]:
for idx in np.argsort(word_feats_normal)[::-1][:10]:
    print(idxfeature(pin, idx), word_feats_normal[idx])

alv 0.2615780602617745
vwl 0.24332700839690996
unr 0.20295134529863534
beg 0.18311407528164095
str 0.17805135380803494
end 0.14587022873916125
fnt 0.14156226241928443
vls 0.14133350243955328
stp 0.13737183743924425
frc 0.10895491590756073


So: lots of vowels, mainly unrounded and front; lots of alveolar stops, more voiced than voiceless. If you were to mimic the sound of this text, in other words, you might say something like "dee dee dat dittee tee day..."

The following cell compares the phoneme feature probabilities in *each line* of the poem to the phoneme feature probabilities of a poem as a whole, and then prints out the top five phoneme features for each line that occur with higher frequency than they do on average in the entire poem. (This is a complicated bit of code, so I included some comments in-line.)

In [34]:
for line in text.split("\n"):
    # get all words in the line
    line_words = [item.lower() for item in re.findall(r"\b(\w+)\b", line)]
    if len(line_words) == 0:  # skip empty lines
        continue
    # calculate then normalize phoneme feature probabilities for each word
    line_word_feats = np.concatenate([pin.phonemefeatures(word) for word in line_words])
    line_word_feats_normal = line_word_feats.sum(axis=0) / line_word_feats.shape[0]
    # subtract the average of the entire text
    diff = line_word_feats_normal - word_feats_normal
    # print line with top five features
    out = []
    for idx in np.argsort(diff)[::-1][:5]:
        out.append("%s: %+0.3f" % (idxfeature(pin, idx), diff[idx]))
    print(line.strip())
    print(", ".join(out))
    print()

'Twas brillig, and the slithy toves
alv: +0.054, frc: +0.035, vls: +0.022, vcd: +0.020, lbd: +0.015

Did gyre and gimble in the wabe:
vcd: +0.093, stp: +0.050, fnt: +0.043, smh: +0.040, unr: +0.029

All mimsy were the borogoves,
vwl: +0.043, lmd: +0.042, blb: +0.035, lat: +0.030, unr: +0.023

And the mome raths outgrabe.
nas: +0.039, low: +0.038, rnd: +0.031, bck: +0.027, umd: +0.015

"Beware the Jabberwock, my son!
blb: +0.047, nas: +0.021, frc: +0.015, low: +0.011, dnt: +0.004

The jaws that bite, the claws that catch!
vls: +0.125, frc: +0.080, dnt: +0.069, stp: +0.061, low: +0.035

Beware the Jubjub bird, and shun
vcd: +0.070, cnt: +0.050, stp: +0.045, mid: +0.043, blb: +0.037

The frumious Bandersnatch!"
frc: +0.058, vls: +0.041, lbd: +0.038, alv: +0.022, hgh: +0.021

He took his vorpal sword in hand;
apr: +0.070, glt: +0.060, end: +0.029, bck: +0.020, rnd: +0.020

Long time the manxome foe he sought---
nas: +0.049, rnd: +0.042, bck: +0.038, vls: +0.031, blb: +0.022

So rested he b

To illustrate how to interpret this output, let's look at the following excerpt:

    Come to my arms, my beamish boy!
    blb: +0.144, nas: +0.078, smh: +0.077, bck: +0.046, end: +0.029
    
This tells us that this line has comparatively more bilabial sounds (Co*m*e to *m*y ar*m*s *m*y *b*ea*m*ish *b*oy), nasal sounds (all of the `m`s), semi-high back vowel sounds, and ends of words. Likewise:

    One, two! One, two! And through and through
    rnd: +0.064, bck: +0.064, alv: +0.051, str: +0.049, nas: +0.046
    
This shows us that the line has round, back, stressed vowels (i.e., the "oo" sounds in "two" and "through") compared to the rest of the poem. Nice!

### Example: Spelling from phoneme features

The Pincelate class has another method, `.spellfeatures()`, which works like `.spell()` except it takes an array of phoneme features (such as that returned from `.phonemefeatures()`) instead of a list of Arpabet phonemes. You can use this to re-spell phoneme feature arrays that you have manipulated. In the following cell, I get the phoneme feature probability array for the word `pug`, then overwrite the probability of the "voiced" feature for its first phoneme, then respell:

In [35]:
pug = pin.phonemefeatures("pug")
pug[1][pin.featureidx('vcd')] = 1
pin.spellfeatures(pug)

'bugh'

Or, you can spell from completely random feature probabilities:

In [36]:
for i in range(12):
    print(pin.spellfeatures(np.random.uniform(0, 1, size=(12,32))))

gyolide
houle
hcoledhe
hyilde
hyolde
hchaulde
guldy
horkei
hchouldy
galedhe
ghile
gcoledy


... which (weirdly) seems like someone trying to imitate the sound of white noise.

Or, you might want to build up neologism from scratch, specifying their phoneme features by hand. To do this, use Pincelate's `.vectorizefeatures()` method, passing it an array of tuples of phoneme features. It returns a phoneme probability array that you can then send to `.spellfeatures()`.

In [37]:
bee = pin.vectorizefeatures([
    ['beg'], ['blb', 'stp', 'vcd'], ['hgh', 'fnt', 'vwl'], ['end']
])
pin.spellfeatures(bee)

'bue'

As an example, the code in the following cell builds up random five-syllable words from phoneme features (picking places and methods of articulation at random) and spells them out:

In [38]:
for i in range(10):
    feats = [["beg"]]
    for j in range(5):
        place = random.choice(['blb', 'alv', 'vel'])
        voice = random.choice(['vcd', 'vls'])
        feats.append([place, voice, 'stp'])
        vowel_place = random.choice([["fnt", "unr"], ["bck", "rnd"]])
        feats.append(vowel_place + ["hgh", "vwl"])
    feats.append(["end"])
    word_feats = pin.vectorizefeatures(feats)
    print(pin.spellfeatures(word_feats))

pubent
piegeaun
toughtee
coughte
ducheis
kidule
giechough
kibune
dibone
guetien


### Example: Resizing feature probability arrays

Once you have the phonetic feature probability arrays, you can treat them the same way you'd treat any other numpy array. One thing I like to do is use scipy's image manipulation functions and use them resample the phonetic feature arrays. This lets us use the same phonetic information to spell a shorter or longer word. In particular, `scipy.ndimage.interpolation` has a handy [zoom](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.interpolation.zoom.html) function that resamples an array and interpolates it. Normally you'd use this to resize an image, but nothing's stopping us from using it to resize our phonetic feature array.

First, import the function:

In [39]:
from scipy.ndimage.interpolation import zoom

Then get some phoneme feature probabilities:

In [40]:
feats = pin.phonemefeatures("alphabet")

Then resize with `zoom()`. The second parameter to `zoom()` is a tuple with the factor by which to scale the dimensions of the incoming array. We only want to scale along the first axis (i.e., the phonemes), keeping the second axis (i.e., the features) constant.

A shorter version of the word:

In [41]:
shorter = zoom(feats, (0.67, 1))
pin.spellfeatures(shorter)

/home/oryx/anaconda3/envs/pincelate/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


'alse'

A longer version:

In [42]:
longer = zoom(feats, (2.0, 1))
pin.spellfeatures(longer)

'alached'

If you've downloaded this notebook and you're following along running the code, the following cell will create an interactive widget that lets you "stretch" and "shrink" the words that you type into the text box by dragging the slider.

In [43]:
import warnings
warnings.filterwarnings('ignore')
@interact(words="how to spell expressively", factor=(0.1, 4.0, 0.1))
def stretchy(words, factor=1.0):
    out = []
    for word in words.split():
        word = word.lower()
        vec = pin.phonemefeatures(word)
        if factor < 1.0:
            order = 3
        else:
            order = 0
        zoomed = zoom(vec, (factor, 1), order=order)
        out.append(pin.spellfeatures(zoomed))
    print(" ".join(out))

interactive(children=(Text(value='how to spell expressively', description='words'), FloatSlider(value=1.0, des…

## Round-trip spelling manipulation

Pincelate actually consists of *two* models: one that knows how to sound out words based on how they're spelled, , and another that knows how to spell words from sounds. Pincelate's `.manipulate()` function does a "round trip" re-spelling of a word, passing it through both models to return back to the original word. Try it out:

In [44]:
pin.manipulate("spelling")

'spenti'

On the surface, this isn't very interesting! You don't need Pincelate to tell you how to spell a word that you already know how to spell. But the `.manipulate()` has a handful of parameters that allow you to mess around with the model's internal workings in fun and interesting ways. The first is the `temperature` parameter, which artificially increases or decreases the amount of randomness in the model's output probabilities.

### Spelling temperature

When the temperature is close to zero, the model will always pick the most likely spelling of the word at each step.

In [45]:
pin.manipulate("spelling", temperature=0.01)

'spean'

As you increase the temperature to 1.0, the model starts picking values at random according to the underlying probabilities.

In [46]:
pin.manipulate("spelling", temperature=1.0)

'spien'

At temperatures above 1.0, the model has a higher chance of picking from letters with lower probabilities, producing a more unlikely spelling:

In [47]:
pin.manipulate("spelling", temperature=1.5)

'spedmje'

At a high enough temperature, the model's spelling feels essentially random:

In [48]:
pin.manipulate("spelling", temperature=3.0)

'bl.'

The following interactive widget lets you play with the `temperature` parameter:

In [49]:
@interact(s="your text here", temp=(0.05, 2.5, 0.05))
def tempadjust(s, temp):
    return ' '.join([pin.manipulate(w.lower(), temperature=temp) for w in s.split()])

interactive(children=(Text(value='your text here', description='s'), FloatSlider(value=1.2500000000000002, des…

### Example: Manipulating letter frequencies

In [50]:
pin.manipulate("spelling", letters={'e': 10})

'spani'

### Example: Manipulating sounds

In [51]:
pin.manipulate("spelling", features={'nas': -10})

'smange'

### Interactive manipulation tool

In [52]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive_output, Layout, HBox, VBox

In [53]:
def manipulate(instr="allison", temp=0.25, **kwargs):
    return ' '.join([
        pin.manipulate(
            w,
            letters={k: v*-1 for k, v in kwargs.items()
                  if k in pin.orth2phon.src_vocab_idx_map.keys()},
            features={k: v*-1 for k, v in kwargs.items()
                      if k in pin.orth2phon.target_vocab_idx_map.keys()},
            temperature=temp
        ) for w in instr.split()]
    )

In [54]:
orth_sliders = {}
phon_sliders = {}
for ch in pin.orth2phon.src_vocab_idx_map.keys():
    if ch in "'-.": continue
    orth_sliders[ch] = widgets.FloatSlider(description=ch,
                               continuous_update=False,
                               value=0,
                               min=-20,
                               max=20,
                               step=0.5,
                               layout=Layout(height="10px"))
for feat in pin.orth2phon.target_vocab_idx_map.keys():
    if feat in ("beg", "end", "cnt", "dnt"): continue
    phon_sliders[feat] = widgets.FloatSlider(description=feat,
                               continuous_update=False,
                               value=0,
                               min=-20,
                               max=20,
                               step=0.5,
                               layout=Layout(height="10px"))
instr = widgets.Text(description='input', value="spelling words with machine learning")
tempslider = widgets.FloatSlider(description='temp', continuous_update=False, value=0.3, min=0.01, max=5, step=0.05)
left_box = VBox(tuple(orth_sliders.values()) + (tempslider,))
right_box = VBox(tuple(phon_sliders.values()))
all_sliders = HBox([left_box, right_box])

out = interactive_output(lambda *args, **kwargs: print(manipulate(*args, **kwargs)),
                         dict(instr=instr, temp=tempslider, **orth_sliders, **phon_sliders))
out.layout.height = "100px"
display(VBox([all_sliders, instr]), out)

Output(layout=Layout(height='100px'))

## Phonetic states

### Example: Homotopies (blending words)

In [55]:
pairs = [('paper', 'plastic'),
         ('kitten', 'puppy'),
         ('birthday', 'anniversary'),
         ('artificial', 'intelligence'),
         ('allison', 'parrish'),
         ('moses', 'middletown'),
         ('day', 'night'),
         ('january', 'december')]

In [56]:
for start_s, end_s in pairs:
    start = pin.phonemestate(start_s)
    end = pin.phonemestate(end_s)
    steps = 2
    out = []
    for i in range(steps+1):
        out.append(
            pin.spellstate(
                (start*(1-(i/steps))) + (end*(i/steps))
            )
        )
    print(" → ".join(out))

peter → prace → place
kiter → coter → putie
birse → ancher → ansear
archain → ancher → inties
allis → allis → paris
mossear → mossear → mitson
dais → dean → nite
janes → deasse → desmar


In [57]:
centroid = (o2ps_tr.translate("wordhack") + \
           o2ps_tr.translate("open") + \
           o2ps_tr.translate("projector")) / 3

NameError: name 'o2ps_tr' is not defined

In [58]:
ps2o_tr.translate(centroid, temp=0.55)

NameError: name 'ps2o_tr' is not defined

### Example: Phonetic resizing of texts

In [59]:
@interact(s="your text here", factor=(0.00, 4, 0.1), continuous_update=False)
def resizer(s, factor=1.0):
    orig = np.array(
        [pin.phonemestate(tok.lower()) for tok in s.split()]
    )
    resized = zoom(orig, (factor, 1), order=4)
    return " ".join([pin.spellstate(vec) for vec in resized])

interactive(children=(Text(value='your text here', description='s'), FloatSlider(value=1.0, description='facto…

### Example: Phonetic similarity with phoneme states

In [60]:
from scipy.spatial.distance import cosine

In [61]:
def similarity(s1, s2):
    return 1 - cosine(pin.phonemestate(s1), pin.phonemestate(s2))

In [62]:
similarity("hello", "bellow")

0.706063449382782

In [63]:
similarity("kiki", "bouba")

0.7466654181480408

In [64]:
similarity("righter", "writer")

0.9869937896728516

In [65]:
similarity("this", "that")

0.974277138710022

In [66]:
similarity("moop", "poom")

0.8288887143135071

In [67]:
similarity("lipstick", "plastic")

0.8154409527778625

In [68]:
similarity("lipstick", "mascara")

0.7981315851211548